In [1]:
import random
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split
from LoadFaceData import load_dataset, resize_image, IMAGE_SIZE

In [2]:
class Dataset:
    def __init__(self, path_name):
        # Test dataset
        self.train_images = None
        self.train_labels = None

        # Validation dataset
        self.valid_images = None
        self.valid_labels = None

        # Test dataset
        self.test_images = None
        self.test_labels = None

        # The path of dataset
        self.path_name = path_name

        # The shape of input data of CNN or LSTM
        self.input_shape = None
    
        #The number of dataset label.
        self.nb_classes = None

    # Load the dataset and preprocess pictures into suitable dim for CNN or LSTM
    def load(self, img_rows=IMAGE_SIZE, img_cols=IMAGE_SIZE,
             img_channels=3):
        
        # Define some useful variables
        image = []
        labels = []
        face_num = []
        
        #Load basical dataset and return it into variables
        images, labels, face_num = load_dataset(self.path_name)
        self.nb_classes = face_num
        
        # Use the split function to separate the basical picture into different array for furture test and validation
        train_images, valid_images, train_labels, valid_labels = train_test_split(images, labels, test_size=0.3,
                                                                                  random_state=random.randint(0, 100))
        _, test_images, _, test_labels = train_test_split(images, labels, test_size=0.5,
                                                          random_state=random.randint(0, 100))

        # if the data dim is 'th'，the data qualitu is ：channels,rows,cols，if not:rows,cols,channels
        # Reshape all dataset including training, validation, test images into a special shape according the keras specification.
        if K.image_data_format() == 'channels_first':
            train_images = train_images.reshape(train_images.shape[0], img_channels, img_rows, img_cols)
            valid_images = valid_images.reshape(valid_images.shape[0], img_channels, img_rows, img_cols)
            test_images = test_images.reshape(test_images.shape[0], img_channels, img_rows, img_cols)
            self.input_shape = (img_channels, img_rows, img_cols)
        else:
            train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, img_channels)
            valid_images = valid_images.reshape(valid_images.shape[0], img_rows, img_cols, img_channels)
            test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, img_channels)
            self.input_shape = (img_rows, img_cols, img_channels)

            # Show the number of training, valid, test image.
            print(train_images.shape[0], 'train samples')
            print(valid_images.shape[0], 'valid samples')
            print(test_images.shape[0], 'test samples')
            #print(dataset.input_shape)
            
            # Convert the labels into one-hot code.
            train_labels = utils.to_categorical(train_labels, self.nb_classes)
            valid_labels = utils.to_categorical(valid_labels, self.nb_classes)
            test_labels = utils.to_categorical(test_labels, self.nb_classes)
            
            # Change the data into float type.
            train_images = train_images.astype('float32')
            valid_images = valid_images.astype('float32')
            test_images = test_images.astype('float32')

            # Nomalization all data in to 0 - 1.
            train_images /= 255
            valid_images /= 255
            test_images /= 255

            # Assign all image and label data to class variables
            self.train_images = train_images
            self.valid_images = valid_images
            self.test_images = test_images
            self.train_labels = train_labels
            self.valid_labels = valid_labels
            self.test_labels = test_labels


# CNN model
class Model:
    def __init__(self):
        self.model = None

    # Built a model
    def build_model(self, dataset, nb_classes):
       # Use a list to discribe a CNN layers.
        # Here contains 4 layers, first layter contain 32 cells, second layer contain 64 cells, third layer 128 cells.
        # The the convolution kernel size is [3,3], padding is same, each layer have the activation is "ReLU"
        # A MaxPooling layer follow each convolution layer.
        # The dense layer followed by a dropout layer, which used to decrease over-fit.Here it give up 25% cells.
        # Use a Flatten layer to conbine all layers output.with a 256 neurons CNN(256 kernels) layer to train the net.
        # Use a dense layer to output a face number classes.

        Conv_layers = [
        
        Convolution2D(filters=32, kernel_size = [3,3], padding = 'same', input_shape = dataset.input_shape, activation = tf.nn.relu),
        MaxPooling2D(pool_size=[2,2]),
        
        Convolution2D(filters=64, kernel_size = [3,3], padding = 'same', input_shape = dataset.input_shape, activation = tf.nn.relu),
        MaxPooling2D(pool_size=[2,2]),
        
        Convolution2D(filters=128, kernel_size = [3,3], padding = 'same', input_shape = dataset.input_shape, activation = tf.nn.relu),
        MaxPooling2D(pool_size=[2,2]),
        
        Convolution2D(filters=256, kernel_size = [3,3], padding = 'same', input_shape = dataset.input_shape, activation = tf.nn.relu),
        MaxPooling2D(pool_size=[2,2]),
            
        Flatten(),
        #Dense(512, activation = tf.nn.relu),
        #Dropout(0.25),
        Dense(256, activation = tf.nn.relu),
        Dropout(0.25),
        Dense(256, activation = tf.nn.relu),
        Dropout(0.25),    
        Dense(nb_classes, activation = tf.nn.softmax)
        
        ]
        self.model = Sequential(Conv_layers)
        self.model.summary()
        
        
        
        
    # Train the CNN model
    def train(self, dataset, batch_size=20, nb_epoch=2, data_augmentation=True):
        
        # Define a SGD optimizer.
        sgd = SGD(learning_rate=0.0007, decay=1e-6,
                  momentum=0.9, nesterov=True)
        
        # Define a Adam optimizer
        adam = Adam(learning_rate = 0.0007, decay = 1e-6)
        
        
        # compile a model, loss funtion use the categorical_crossentropy.
        #self.model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
        self.model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Without using data boosting, by which we mean creating new training data from the training data we provide using methods such as rotating, flipping, adding noise, etc.
        # Training data, consciously boosting the size of the training data to increase the amount of model training
        if not data_augmentation:
            self.model.fit(dataset.train_images,
                           dataset.train_labels,
                           batch_size=batch_size,
                           nb_epoch=nb_epoch,
                           validation_data=(dataset.valid_images, dataset.valid_labels),
                           shuffle=True)
        # Use real-time data to enhance
        else:
            # Define a data generator for data lifting, which returns a generator object datagen, which is called once every
            # It generates a set of data (sequentially) every time it is called, saving memory, in fact it is python's data generator
            datagen = ImageDataGenerator(
                featurewise_center=False,  # Whether to decentralize the input data (mean 0)
                samplewise_center=False,  # Whether to make the mean of each sample of the input data 0
                # Whether the data is normalised (input data divided by the standard deviation of the data set)
                featurewise_std_normalization=False,  
                samplewise_std_normalization=False,  # Whether to divide each sample data by its own standard deviation
                zca_whitening=False,  # Whether to apply ZCA whitening to the input data
                rotation_range=20,  # The angle at which the picture is randomly rotated when the data is lifted (range 0 to 180)
                # The magnitude of the horizontal offset of the image when the data is lifted 
                # (in % of the image width, floating point number between 0 and 1)
                width_shift_range=0.2,  
                height_shift_range=0.2,  # Same as above, except here it's vertical
                horizontal_flip=True,  # Whether to perform random horizontal flipping
                vertical_flip=False)  # Whether to perform a random vertical flip

            # Calculate the number of samples in the entire training set for eigenvalue normalization, ZCA whitening, etc.
            datagen.fit(dataset.train_images)

            # Start training the model with the generator
            self.model.fit_generator(datagen.flow(dataset.train_images, dataset.train_labels,
                                                  batch_size=batch_size),
                                     validation_data=(dataset.valid_images, dataset.valid_labels),
                                     steps_per_epoch=dataset.train_images.shape[0],
                                     epochs=nb_epoch)
            '''self.model.fit(datagen.flow(dataset.train_images, dataset.train_labels,
                                                  batch_size=batch_size),
                                     validation_data=(dataset.valid_images, dataset.valid_labels),
                                     steps_per_epoch=len(dataset.train_images)/20,
                                     epochs=nb_epoch)'''
    # Define a model parameter store path
    MODEL_PATH = './model'

    # Save the model parameter in the defined path
    def save_model(self, file_path=MODEL_PATH):
        self.model.save(file_path)

    #Load a model from the defined path
    def load_model(self, file_path=MODEL_PATH):
        self.model = load_model(file_path)

    # Evaluate a model from the defined path using the dataset
    def evaluate(self, dataset):
        score = self.model.evaluate(dataset.test_images, dataset.test_labels, verbose=1)
        print("%s: %.2f%%" % (self.model.metrics_names[1], score[1] * 100))
    
    # Recognize the face
    def face_predict(self, image):

        # Accordinf the Dim to reshape the image shape.
        if K.image_data_format() == 'channels_first':
            image = resize_image(image)  # 尺寸必须与训练集一致都应该是IMAGE_SIZE x IMAGE_SIZE
            image = image.reshape((1, 3, IMAGE_SIZE, IMAGE_SIZE))  # 与模型训练不同，这次只是针对1张图片进行预测
        else:
            image = resize_image(image)
            image = image.reshape((1, IMAGE_SIZE, IMAGE_SIZE, 3))
            
   
       # Float and normalize the input.
        image = image.astype('float32')
        image /= 255
        
        # Print the probability of name response the present face.
        result_probability = self.model.predict(image)
        print('result:', result_probability, max(result_probability[0]))

        # Output the one-hot code and change it into the name od user.
        result = self.model.predict(image)
        result = np.argmax(result, axis=-1)

        # 返回类别预测结果
        return max(result_probability[0]),result[0]


if __name__ == '__main__':
    dataset = Dataset('./New Face data')
    dataset.load()
    model = Model()
    model.build_model(dataset, dataset.nb_classes)
    model.train(dataset)
    model.save_model(file_path='./model') 
    model.evaluate(dataset)


(5782, 64, 64, 3)
['Yuxi Wu', 'Xingda Zhou', 'Yuan Li', 'Zhongen Qin', 'Jufeng Yang', 'Jiahao Wang']
face_num: 6
contrast_table: {0: 'Yuxi Wu', 1: 'Xingda Zhou', 2: 'Yuan Li', 3: 'Zhongen Qin', 4: 'Jufeng Yang', 5: 'Jiahao Wang'}
4047 train samples
1735 valid samples
2891 test samples
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None

accuracy: 99.72%
